In [1]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2015-03-24 11:55:43--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net... 54.192.101.25, 54.230.101.178, 54.230.101.43, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|54.192.101.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: 'phix.fa.1'

100%[======================================>] 5,528       --.-K/s   in 0s      

2015-03-24 11:55:43 (57.3 MB/s) - 'phix.fa.1' saved [5528/5528]



In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('phix.fa')

In [4]:
def NaiveExactMatching(genome, read):
    # Keep a list of all indices where the read matches
    matches = []
    num_comparisons = 0
    for i in range(len(genome)-len(read)+1):
        match = True
        for j in range(len(read)):
            num_comparisons += 1
            if not read[j] == genome[i+j]:
                match = False
                break
        if match:
            matches.append(i)

    #print('\n%d total comparisons' % num_comparisons)
    return matches

In [5]:
import random
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

In [6]:
# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# Count how many reads match the genome exactly
numMatched = 0
for r in reads:
    matches = NaiveExactMatching(genome, r)
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


In [7]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2015-03-24 11:55:44--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net... 54.192.101.8, 54.230.101.178, 54.230.101.43, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|54.192.101.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: 'ERR266411_1.first1000.fastq'

100%[======================================>] 254,384     --.-K/s   in 0.09s   

2015-03-24 11:55:44 (2.79 MB/s) - 'ERR266411_1.first1000.fastq' saved [254384/254384]



In [8]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

In [9]:
import collections
phix_reads = readFastq('ERR266411_1.first1000.fastq')
count = collections.Counter()
for read in phix_reads:
    count.update(read)
count

Counter({'T': 30531, 'A': 28426, 'C': 21890, 'G': 19147, 'N': 6})

In [10]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = NaiveExactMatching(genome, r)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

459 / 1000 reads matched the genome exactly!


In [11]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [12]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = NaiveExactMatching(genome, r)
    matches.extend(NaiveExactMatching(genome, reverseComplement(r)))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

932 / 1000 reads matched the genome exactly!
